In [ ]:
import torch
from torchvision.datasets import VOCSegmentation
import torchvision.transforms.v2 as transforms
import torchvision.transforms.functional as F

# Load image and segmentation mask
IMG_SIZE = (256, 256)
NUM_CLASSES = 21 # 20 classes + the background

# Transformation class
class VOCTransforms:
    def __init__(self, img_size, mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)):
        self.transform = transforms.Compose([
            transforms.Resize(img_size, interpolation=transforms.InterpolationMode.BILINEAR)
            transforms.ToImage(),
            transforms.ToDtype(torch.long, scale=False),
        ])
    
    def __call__(self, img, target):
        image = self.transform(img)
        target = self.target_transform(target)
        target = target.squeeze(0)
        return image, target

# Preprocess image (resize, normalize)

# Define U-Net model:
    # Encoder path (downsampling):
        # conv → relu → conv → relu → maxpool
        # repeat, doubling channels

    # Decoder path (upsampling):
        # upsample → concat with encoder feature map → convs
        # repeat, halving channels

    # Final 1x1 conv → class scores for each pixel

# Forward pass → compute loss → backprop → update

# Use trained model to predict segmentation masks
